In [1]:
import os
import pickle
import numpy as np
import PIL.Image
import dnnlib
import dnnlib.tflib as tflib
import config
import tensorflow as tf

tflib.init_tf()
url = 'https://drive.google.com/uc?id=1MEGjdvVpUsu1jB4zrXZN7Y4kBBOzizDQ' # karras2019stylegan-ffhq-1024x1024.pkl
with dnnlib.util.open_url(url, cache_dir=config.cache_dir) as f:
    _G, _D, Gs = pickle.load(f)


Instructions for updating:
Colocations handled automatically by placer.


In [4]:
image_folder = './experiments/aligned_images/'
image_name = 'white_man_01.png'
image_path = image_folder + image_name
output_folder = './experiments/L2_z/'
image_size = (1024, 1024)
learning_rate = 0.001
num_iter = 1000

# Create target image variable 
image = PIL.Image.open(image_path)
image.resize(image_size, PIL.Image.ANTIALIAS)
image = np.array(image, dtype=np.float32)
with tf.variable_scope("recovery_scope", reuse=tf.AUTO_REUSE):
    target_image_variable = tf.get_variable('target_image', 
                                            dtype=tf.float32,
                                            initializer=tf.constant(image))
print(target_image_variable)
target_variable = tf.expand_dims(target_image_variable, 0)
# target_variable = tflib.convert_images_from_uint8(target_variable, drange=[-1,1], nhwc_to_nchw=True)
print(target_variable)

# Create initial latent variable 
# z = tf.Variable(np.random.randn(1, Gs.input_shape[1]), dtype=tf.float32)
with tf.variable_scope("recovery_scope", reuse=tf.AUTO_REUSE):
    z = tf.get_variable('learnable_latents',
                        shape=(1, Gs.input_shape[1]),
                        dtype=tf.float32,
                        initializer=tf.initializers.random_normal())
print(z)

# Create output image variable
# output_variable = Gs.get_output_for(z, None, is_validation=True, randomize_noise=False)
output_variable = Gs.get_output_for(z, None, truncation_psi=0.7, randomize_noise=False)
# formatted_output_variable = tflib.convert_images_to_uint8(output_variable, drange=[-1,1], nchw_to_nhwc=True)
formatted_output_variable = tflib.convert_images_to_uint8(output_variable, 
                                                          drange=[-1,1], 
                                                          nchw_to_nhwc=True, 
                                                          uint8_cast=False)
formatted_output_variable_uint8 = tf.saturate_cast(formatted_output_variable, tf.uint8)


print(output_variable)
print(formatted_output_variable)

# Loss
# loss = tf.losses.mean_squared_error(labels=target_variable, predictions=output_variable)
loss = tf.losses.mean_squared_error(labels=target_variable, predictions=formatted_output_variable)
print(loss)

# Optimizer
optimizer = tf.train.GradientDescentOptimizer(learning_rate)
# optimizer = tf.train.AdamOptimizer(learning_rate)
train_op = optimizer.minimize(loss, var_list=z)
print(optimizer)
print(train_op)

# Session
sess = tf.get_default_session()
# sess.run(tf.global_variables_initializer())
sess.run(z.initializer)
sess.run(target_image_variable.initializer)
formatted_output_value_uint8 = None
z_value = None
if not os.path.exists(output_folder + image_name):
    os.mkdir(output_folder + image_name)
for it in range(num_iter):
    _, loss_value, z_value, formatted_output_value_uint8, formatted_output_value, target_value = sess.run((train_op, loss, z, formatted_output_variable_uint8, formatted_output_variable, target_variable))
    print("-------")    
    print(it)
    PIL.Image.fromarray(formatted_output_value_uint8[0], 'RGB').save((output_folder + image_name + '/{}.png').format(it))
    print("Loss: ", loss_value)
#     print("formatted_output_value: ", formatted_output_value)
#     print("target_value: ", target_value)

# Save image
PIL.Image.fromarray(formatted_output_value_uint8[0], 'RGB').save(output_folder + image_name)
np.save(os.path.join('./results/recovery.npy'), z_value)


<tf.Variable 'recovery_scope/target_image:0' shape=(1024, 1024, 3) dtype=float32_ref>
Tensor("ExpandDims_2:0", shape=(1, 1024, 1024, 3), dtype=float32)
<tf.Variable 'recovery_scope/learnable_latents:0' shape=(1, 512) dtype=float32_ref>
Tensor("Gs_3/images_out:0", shape=(1, 3, 1024, 1024), dtype=float32)
Tensor("add_2:0", shape=(1, 1024, 1024, 3), dtype=float32)
Tensor("mean_squared_error_2/value:0", shape=(), dtype=float32)
name: "GradientDescent_2"
op: "NoOp"
input: "^GradientDescent_2/update_recovery_scope/learnable_latents/ApplyGradientDescent"



UnknownError: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node Gs_3/G_synthesis/4x4/Conv/Conv2D (defined at <string>:159) ]]
	 [[node add_2 (defined at /home/ubuntu/cs230/stylegan-encoder/dnnlib/tflib/tfutil.py:239) ]]

Caused by op 'Gs_3/G_synthesis/4x4/Conv/Conv2D', defined at:
  File "/home/ubuntu/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/ubuntu/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/ubuntu/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 486, in start
    sel

In [ ]:
# A minimum example to run 
z = tf.Variable(np.random.randn(1, Gs.input_shape[1]), dtype=tf.float32)
output_variable = Gs.get_output_for(z, None, truncation_psi=0.7, randomize_noise=False)
formatted_output_variable = tflib.convert_images_to_uint8(output_variable, drange=[-1,1], nchw_to_nhwc=True)
print(output_variable)
sess = tf.get_default_session()
in_expr = [z]
mb_in = [np.random.randn(1, Gs.input_shape[1])]
formmated_output_value = sess.run(formatted_output_variable, feed_dict=dict(zip(in_expr, mb_in)))
PIL.Image.fromarray(formmated_output_value[0], 'RGB').save('./results/recovery.png')


In [ ]:
print(Gs)

In [ ]:
from functools import partial

def create_stub(name, batch_size):
    return tf.constant(0, dtype='float32', shape=(batch_size, 0))


def create_variable_for_generator(name, batch_size):
    return tf.get_variable('learnable_dlatents',
                           shape=(batch_size, 18, 512),
                           dtype='float32',
                           initializer=tf.initializers.random_normal())


initial_dlatents = np.zeros((1, 18, 512))
Gs.components.synthesis.run(initial_dlatents,
                               randomize_noise=False, minibatch_size=1,
                               custom_inputs=[partial(create_variable_for_generator, batch_size=1),
                                              partial(create_stub, batch_size=1)],
                               structure='fixed')

sess = tf.get_default_session()
graph = tf.get_default_graph()




In [ ]:
vs = tf.global_variables()
for v in vs:
    print(v)
    
Gs.print_layers()

In [ ]:
optimizer = tf.train.AdamOptimizer(learning_rate)

In [ ]:
print(sess.run(tf.report_uninitialized_variables()))